In [56]:
import os
import pandas as pd

import requests
from bs4 import BeautifulSoup

import re

path = os.path.abspath(os.path.dirname(os.getcwd())) + '\\data'

In [64]:
issue_df = pd.read_csv(path+'\\website\\issue.csv')

def find_website_article(i):
    
    number_volume = issue_df["number_volume"].iloc[i]
    number_issue = issue_df["number_issue"].iloc[i]
    
    url_issue = issue_df["website_issue"].iloc[i]
    response = requests.get(url=url_issue)
    
    web_content = response.content
    bs_obj=BeautifulSoup(web_content,'lxml').body.div.div
    
    article = bs_obj.find_all(name='div', attrs={"class":"articleEntry"})
    
    article_df = pd.DataFrame(columns=["number_volume","number_issue","number_article","type","link"])
    
    for i in range(len(article)):
        try:
            type = article[i].find(name='div', attrs={"class":"article-type"}).string
        except:
            continue
            
        link_str = str(article[i].find(name='div', attrs={"class":"art_title linkable"}))
        link = re.findall(r"href=\"(.+?)\">", link_str)[0]
        link = 'https://www.tandfonline.com' + link
        
        article_df.loc[i] = [number_volume, number_issue, i+1, type, link]
    
    return article_df

In [68]:
num_issue = issue_df.shape[0]

article_df_all = pd.DataFrame(columns=["number_volume","number_issue","number_article","type","link"])

for i_issue in range(num_issue):
    
    article_df = find_website_article(i_issue)
    
    article_df_all = pd.concat([article_df_all, article_df], axis=0, ignore_index=True)

In [70]:
article_df_all.to_csv(path+'\\website_article\\article.csv',index=False,sep=',')